# Ejercicio 1

# Borradores

## Obteniendo secuencias sobre la carpeta de genomas

Importando librerias e iniciando SparkSession

In [1]:
import time
import pandas as pd
import time
import glob
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, length
from pyspark.sql.types import StringType, IntegerType

In [2]:
# Inicializar Spark
spark = SparkSession.builder.appName("FASTA_Kmer_Matching").getOrCreate()
sc = spark.sparkContext


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/10 17:21:14 WARN Utils: Your hostname, Felipes-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.12 instead (on interface en0)
25/07/10 17:21:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/10 17:21:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Obtención de paths

In [10]:
path_obj = os.path.join(os.getcwd(), 'files', 'genome_ref')
files_path = os.path.join(os.getcwd(), 'files', 'genomes')
fasta_obj = os.path.join(path_obj, 'GCF_003119375.1_ASM311937v1_genomic.fna')

In [11]:
df_path = pd.DataFrame(os.listdir(files_path), columns=['filename'])
df_path

,filename
0,GCF_902702785.1_GD-0001_genomic.fna
1,GCF_902702755.1_S10_genomic.fna


In [12]:
records = []

for file in df_path['filename']:
    if not file.endswith('.fna'):
        continue

    print(f"Processing file: {file}")
    file_path = os.path.join(files_path, file)
    with open(file_path) as f:
        header = f.readline().strip()
        genome = "".join(line.strip() for line in f if not line.startswith(">"))

    # Acumula en lista
    records.append({
        'filename': file,
        'seq_header': header,
        'genome': genome
    })

df_genome = pd.DataFrame(records)
df_genome

Processing file: GCF_902702785.1_GD-0001_genomic.fna
Processing file: GCF_902702755.1_S10_genomic.fna


,filename,seq_header,genome
0,GCF_902702785.1_GD-0001_genomic.fna,>NZ_LR738720.1 Streptococcus suis isolate GD-0...,ATGAACCAAGAACAACTTTTTTGGCAACGATTTATTGAATTGGCAA...
1,GCF_902702755.1_S10_genomic.fna,>NZ_LR738721.1 Streptococcus suis isolate S10 ...,ATGAACCAAGAACAACTTTTTTGGCAACGATTTATTGAATTGGCAA...


In [6]:
reads_rdd = sc.parallelize(df_genome['genome'].tolist())

## Obteniendo secuencias del archivo objetivo

In [15]:
with open(fasta_obj) as f:
        header = f.readline().strip()
        genome_obj = "".join(line.strip() for line in f if not line.startswith(">"))

genome_obj[:10]  # primeros 100 chars

'TTGGAGGTTG'

In [17]:
genome_obj_bc = sc.broadcast(genome_obj)

In [7]:
# Parámetro k
k = 2
# Función para obtener k-mers de longitud k
def kmers(seq: str, k: int):
    return [ seq[i : i + k] 
             for i in range(len(seq) - k + 1) ]

In [21]:
# Función para contar hits de un k-mer en el genoma
def find_kmers_in_genome(genome: str, kmer: str) -> list[int]:
    positions = []
    pos = genome.find(kmer)  # busca la 1ª aparición
    while pos != -1:
        positions.append(pos)  # guardamos la posición
        pos = genome.find(kmer, pos + 1)  # buscamos la siguiente (permite solapamientos)
    return positions

In [ ]:
def process_read(record):
    header, seq = record
    kmer_list = kmers(seq, k)
    t0 = time.time()
    sum_cal = 0
    all_pos = []
    for mer in kmer_list:
        positions = find_kmers_in_genome(genome_obj_bc.value, mer)
        if positions:
            sum_cal += len(positions)
            all_pos.extend(positions)
    dt = time.time() - t0
    return {"Sequence": header, "Sum_Cal": sum_cal, "Positions": all_pos, "Time(s)": dt}

In [ ]:
# Ejecutar en paralelo y recolectar resultados
rows = reads_rdd.map(process_read).collect()

# Construir DataFrame y exportar
df = pd.DataFrame(rows)
df.to_csv('scripts/results/output.csv', index=False, encoding='utf-8')

print("El archivo output.csv ha sido creado con éxito.")

In [2]:
import time
import pandas as pd
import time
import glob
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, length
from pyspark.sql.types import StringType, IntegerType

# Inicializar Spark
spark = SparkSession.builder.appName("FASTA_Kmer_Matching").getOrCreate()
sc = spark.sparkContext

path_obj = os.path.join(os.getcwd(), 'files', 'genome_ref')
files_path = os.path.join(os.getcwd(), 'files', 'genomes')
fasta_obj = os.path.join(path_obj, 'GCF_003119375.1_ASM311937v1_genomic.fna')

df_path = pd.DataFrame(os.listdir(files_path), columns=['filename'])

records = []

for file in df_path['filename']:
    if not file.endswith('.fna'):
        continue

    print(f"Processing file: {file}")
    file_path = os.path.join(files_path, file)
    with open(file_path) as f:
        header = f.readline().strip()
        genome = "".join(line.strip() for line in f if not line.startswith(">"))

    # Acumula en lista
    records.append({
        'filename': file,
        'seq_header': header,
        'genome': genome
    })

with open(fasta_obj) as f:
        header = f.readline().strip()
        genome_obj = "".join(line.strip() for line in f if not line.startswith(">"))

# Tamaño de k-mers
k = 2
def kmers(seq: str, k: int):
    return [ seq[i : i + k] 
             for i in range(len(seq) - k + 1) ]

# Función para contar hits de un k-mer en el genoma
def find_kmers_in_genome(genome: str, kmer: str) -> list[int]:
    positions = []
    pos = genome.find(kmer)  # busca la 1ª aparición
    while pos != -1:
        positions.append(pos)  # guardamos la posición
        pos = genome.find(kmer, pos + 1)  # buscamos la siguiente (permite solapamientos)
    return positions

def process_read(seq: str):
    kmer_list = kmers(seq, k)
    t0 = time.time()
    sum_cal = 0
    all_pos = []
    for mer in kmer_list:
        positions = find_kmers_in_genome(genome_obj_bc.value, mer)
        if positions:
            sum_cal += len(positions)
            all_pos.extend(positions)
    dt = time.time() - t0
    return {"Sequence": seq[:30] + "...", "Sum_Cal": sum_cal, "Positions": all_pos, "Time(s)": dt}

# Crear DataFrame de Pandas con los registros
df_genome = pd.DataFrame(records)
reads_rdd = sc.parallelize(df_genome['genome'].tolist())

# Broadcast del genoma de referencia
genome_obj_bc = sc.broadcast(genome_obj)


# Ejecutar en paralelo y recolectar resultados
results_rdd = reads_rdd.map(process_read).collect()

# Construir DataFrame y exportar
df = pd.DataFrame(results_rdd)
df.to_csv('scripts/results/output.csv', index=False, encoding='utf-8')

print("El archivo output.csv ha sido creado con éxito.")

Processing file: GCF_902702785.1_GD-0001_genomic.fna
Processing file: GCF_902702755.1_S10_genomic.fna


25/07/10 19:26:02 WARN TaskSetManager: Stage 1 contains a task of very large size (2079 KiB). The maximum recommended task size is 1000 KiB.
ERROR:root:KeyboardInterrupt while sending command.=====>         (10 + 2) / 12]
Traceback (most recent call last):
  File "/Users/felipeespinoza/.pyenv/versions/3.12.5/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/felipeespinoza/.pyenv/versions/3.12.5/lib/python3.12/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/felipeespinoza/.pyenv/versions/3.12.5/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
import time
import pandas as pd
import time
import glob
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, length
from pyspark.sql.types import StringType, IntegerType

# Inicializar Spark
spark = SparkSession.builder.appName("FASTA_Kmer_Matching").getOrCreate()
sc = spark.sparkContext

path_obj = os.path.join(os.getcwd(), 'files', 'genome_ref')
files_path = os.path.join(os.getcwd(), 'files', 'genomes')
fasta_obj = os.path.join(path_obj, 'GCF_003119375.1_ASM311937v1_genomic.fna')

df_path = pd.DataFrame(os.listdir(files_path), columns=['filename'])

records = []

for file in df_path['filename']:
    if not file.endswith('.fna'):
        continue

    print(f"Processing file: {file}")
    file_path = os.path.join(files_path, file)
    with open(file_path) as f:
        header = f.readline().strip()
        genome = "".join(line.strip() for line in f if not line.startswith(">"))

    # Acumula en lista
    records.append({
        'filename': file,
        'seq_header': header,
        'genome': genome
    })

with open(fasta_obj) as f:
        header = f.readline().strip()
        genome_obj = "".join(line.strip() for line in f if not line.startswith(">"))

# Tamaño de k-mers
k = 2
def kmers(seq: str, k: int):
    return [ seq[i : i + k] 
             for i in range(len(seq) - k + 1) ]

def build_kmer_index(genome: str, k: int):
    index = {}
    for i in range(len(genome) - k + 1):
        kmer = genome[i:i+k]
        index.setdefault(kmer, []).append(i)
    return index

def process_read(seq: str):
    kmer_list = kmers(seq, k)
    t0 = time.time()
    sum_cal = 0
    all_pos = []
    for mer in kmer_list:
        positions = genome_index_bc.value.get(mer, [])
        if positions:
            sum_cal += len(positions)
            all_pos.extend(positions)
    dt = time.time() - t0
    return {"Sequence": seq[:30] + "...", "Sum_Cal": sum_cal, "Positions": all_pos, "Time(s)": dt}

# Crear DataFrame de Pandas con los registros
df_genome = pd.DataFrame(records)
reads_rdd = sc.parallelize(df_genome['genome'].tolist())

genome_index = build_kmer_index(genome_obj, k)
genome_index_bc = sc.broadcast(genome_index)

reads_rdd = reads_rdd.repartition(8)  # ajusta el número a los núcleos disponibles

df_results = spark.createDataFrame(reads_rdd.map(process_read))
df_results.toPandas().to_csv('scripts/results/output.csv', index=False, encoding='utf-8')

print("El archivo output.csv ha sido creado con éxito.")

Processing file: GCF_902702785.1_GD-0001_genomic.fna
Processing file: GCF_902702755.1_S10_genomic.fna


25/07/10 19:31:14 WARN TaskSetManager: Stage 2 contains a task of very large size (2079 KiB). The maximum recommended task size is 1000 KiB.


In [1]:
import time
import os
from pyspark.sql import SparkSession, Row

# Inicializar Spark
spark = SparkSession.builder.appName("FASTA_Kmer_Matching").getOrCreate()
sc = spark.sparkContext

path_obj = os.path.join(os.getcwd(), 'files', 'genome_ref')
files_path = os.path.join(os.getcwd(), 'files', 'genomes')
fasta_obj = os.path.join(path_obj, 'GCF_003119375.1_ASM311937v1_genomic.fna')

# Read all FASTA files in parallel and parse into (header, sequence) tuples
files_rdd = sc.wholeTextFiles(os.path.join(files_path, '*.fna'))

def parse_fasta(file_pair):
    _, text = file_pair
    recs = []
    for record in text.split('>'):
        if not record: continue
        lines = record.splitlines()
        header = lines[0]
        seq = ''.join(lines[1:])
        recs.append((header, seq))
    return recs

reads_rdd = files_rdd.flatMap(parse_fasta) \
                     .repartition(8)

with open(fasta_obj) as f:
    header = f.readline().strip()
    genome_obj = "".join(line.strip() for line in f if not line.startswith(">"))

# Tamaño de k-mers
k = 2
def kmers(seq: str, k: int):
    return [ seq[i : i + k] 
             for i in range(len(seq) - k + 1) ]

def build_kmer_index(genome: str, k: int):
    index = {}
    for i in range(len(genome) - k + 1):
        kmer = genome[i:i+k]
        index.setdefault(kmer, []).append(i)
    return index

def process_read(seq: str):
    kmer_list = kmers(seq, k)
    t0 = time.time()
    sum_cal = 0
    all_pos = []
    for mer in kmer_list:
        positions = genome_index_bc.value.get(mer, [])
        if positions:
            sum_cal += len(positions)
            all_pos.extend(positions)
    dt = time.time() - t0
    return {"Sequence": seq[:30] + "...", "Sum_Cal": sum_cal, "Positions": all_pos, "Time(s)": dt}

# Build genome index and broadcast
genome_index = build_kmer_index(genome_obj, k)
genome_index_bc = sc.broadcast(genome_index)

from pyspark.sql import Row

# Compute results in parallel and write out
rows_rdd = reads_rdd.map(lambda rec: process_read(rec[1]) if isinstance(rec, tuple) else process_read(rec))
spark_df = spark.createDataFrame(rows_rdd.map(lambda d: Row(**d)))
spark_df.write.mode("overwrite") \
    .option("header", True) \
    .csv('results/output_csv')

print("Results written to results/output_csv")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/10 20:47:22 WARN Utils: Your hostname, Felipes-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.10 instead (on interface en8)
25/07/10 20:47:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/10 20:47:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
ERROR:root:Exception while sending command.==========>              (3 + 1) / 4]
Traceback (most recent call last):
  File "/Users/felipeespinoza/.pyenv/versions/3.12.5/lib/python3.12/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
              

Py4JError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob

In [3]:
import time
import os
from pyspark.sql import SparkSession, Row

# Inicializar Spark
spark = SparkSession.builder.appName("FASTA_Kmer_Matching").getOrCreate()
sc = spark.sparkContext

path_obj = os.path.join(os.getcwd(), 'files', 'genome_ref')
files_path = os.path.join(os.getcwd(), 'files', 'genomes')
fasta_obj = os.path.join(path_obj, 'GCF_003119375.1_ASM311937v1_genomic.fna')

# Read all FASTA files in parallel and parse into (header, sequence) tuples
files_rdd = sc.wholeTextFiles(os.path.join(files_path, '*.fna'))

def parse_fasta(file_pair):
    _, text = file_pair
    recs = []
    for record in text.split('>'):
        if not record: continue
        lines = record.splitlines()
        header = lines[0]
        seq = ''.join(lines[1:])
        recs.append((header, seq))
    return recs

reads_rdd = files_rdd.flatMap(parse_fasta) \
                     .repartition(54)

with open(fasta_obj) as f:
    header = f.readline().strip()
    genome_obj = "".join(line.strip() for line in f if not line.startswith(">"))

# Tamaño de k-mers
k = 2
def kmers(seq: str, k: int):
    return [ seq[i : i + k] 
             for i in range(len(seq) - k + 1) ]

def build_kmer_index(genome: str, k: int):
    index = {}
    for i in range(len(genome) - k + 1):
        kmer = genome[i:i+k]
        index.setdefault(kmer, []).append(i)
    return index

def process_read(seq: str):
    kmer_list = kmers(seq, k)
    t0 = time.time()
    sum_cal = 0
    all_pos = []
    for mer in kmer_list:
        sum_cal += len(genome_index_bc.value.get(mer, []))
    dt = time.time() - t0
    return {"Sequence": seq[:30] + "...",
             "Sum_Cal": sum_cal,
               "Time(s)": dt}

# Build genome index and broadcast
genome_index = build_kmer_index(genome_obj, k)
genome_index_bc = sc.broadcast(genome_index)

# Compute results in parallel and write out
rows_rdd = reads_rdd.map(lambda rec: process_read(rec[1]) if isinstance(rec, tuple) else process_read(rec))
spark_df = spark.createDataFrame(rows_rdd.map(lambda d: Row(**d)))
spark_df.write.mode("overwrite") \
    .option("header", True) \
    .csv('results/output_csv')

print("Results written to results/output_csv")

Results written to results/output_csv


In [5]:
import time
import os
from pyspark.sql import SparkSession, Row

# Inicializar Spark
spark = SparkSession.builder.appName("FASTA_Kmer_Matching").getOrCreate()
sc = spark.sparkContext

path_obj = os.path.join(os.getcwd(), 'files', 'genome_ref')
files_path = os.path.join(os.getcwd(), 'files', 'genomes')
fasta_obj = os.path.join(path_obj, 'GCF_003119375.1_ASM311937v1_genomic.fna')

# Read all FASTA files in parallel and parse into (header, sequence) tuples
files_rdd = sc.wholeTextFiles(os.path.join(files_path, '*.fna'))

# Summarize each FASTA file: count sequences, total length, GC content
def summarize_fasta(file_pair):
    path, text = file_pair
    lines = text.splitlines()
    num_seq = sum(1 for line in lines if line.startswith('>'))
    seq_lines = [line.strip() for line in lines if not line.startswith('>')]
    total_len = sum(len(seq) for seq in seq_lines)
    gc_count = sum(seq.count('G') + seq.count('C') for seq in seq_lines)
    gc_pct = round(100 * gc_count / total_len, 2) if total_len > 0 else 0.0
    filename = os.path.basename(path)
    return Row(filename=filename,
               num_sequences=num_seq,
               total_length=total_len,
               gc_pct=gc_pct)

# Create Results directory if not exists
os.makedirs("Results", exist_ok=True)
# Create DataFrame of summaries and write CSV
summary_df = spark.createDataFrame(files_rdd.map(summarize_fasta))
summary_df.write.mode("overwrite") \
    .option("header", True) \
    .csv("Results/summary")
print("FASTA summaries written to Results/summary")

def parse_fasta(file_pair):
    _, text = file_pair
    recs = []
    for record in text.split('>'):
        if not record: continue
        lines = record.splitlines()
        header = lines[0]
        seq = ''.join(lines[1:])
        recs.append((header, seq))
    return recs

reads_rdd = files_rdd.flatMap(parse_fasta) \
                     .repartition(54)

with open(fasta_obj) as f:
    header = f.readline().strip()
    genome_obj = "".join(line.strip() for line in f if not line.startswith(">"))

# Tamaño de k-mers
k = 2
def kmers(seq: str, k: int):
    return [ seq[i : i + k] 
             for i in range(len(seq) - k + 1) ]

def build_kmer_index(genome: str, k: int):
    index = {}
    for i in range(len(genome) - k + 1):
        kmer = genome[i:i+k]
        index.setdefault(kmer, []).append(i)
    return index

def process_read(seq: str):
    kmer_list = kmers(seq, k)
    t0 = time.time()
    sum_cal = 0
    all_pos = []
    for mer in kmer_list:
        sum_cal += len(genome_index_bc.value.get(mer, []))
    dt = time.time() - t0
    return {"Sequence": seq[:30] + "...",
             "Sum_Cal": sum_cal,
               "Time(s)": dt}

# Build genome index and broadcast
genome_index = build_kmer_index(genome_obj, k)
genome_index_bc = sc.broadcast(genome_index)

# Summarize each FASTA for k=2 matches in the reference
def summarize_k2(file_pair):
    path, text = file_pair
    lines = text.splitlines()
    num_seq = sum(1 for line in lines if line.startswith('>'))
    seq_lines = [line.strip() for line in lines if not line.startswith('>')]
    total_len = sum(len(s) for s in seq_lines)
    seq_str = "".join(seq_lines)
    # Count all 2-mer matches against the reference index
    k2_count = 0
    for i in range(len(seq_str) - 1):
        k2 = seq_str[i:i+2]
        k2_count += len(genome_index_bc.value.get(k2, []))
    filename = os.path.basename(path)
    return Row(filename=filename,
               num_sequences=num_seq,
               total_length=total_len,
               k2_matches=k2_count)

# Write summary of 2-mer matches
os.makedirs("Results", exist_ok=True)
summary_k2_df = spark.createDataFrame(files_rdd.map(summarize_k2))
summary_k2_df.write.mode("overwrite") \
    .option("header", True) \
    .csv("Results/summary_k2")
print("2-mer match summaries written to Results/summary_k2")

# Compute results in parallel and write out
rows_rdd = reads_rdd.map(lambda rec: process_read(rec[1]) if isinstance(rec, tuple) else process_read(rec))
spark_df = spark.createDataFrame(rows_rdd.map(lambda d: Row(**d)))
spark_df.write.mode("overwrite") \
    .option("header", True) \
    .csv('results/output_csv')

print("Results written to results/output_csv")

FASTA summaries written to Results/summary


2-mer match summaries written to Results/summary_k2


Results written to results/output_csv
